<a href="https://colab.research.google.com/github/MPGarg/Assignment-11/blob/main/Assignment11_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
!git clone https://github.com/MPGarg/main_repo.git

Cloning into 'main_repo'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 131 (delta 33), reused 47 (delta 21), pack-reused 52
Receiving objects: 100% (131/131), 49.38 KiB | 16.46 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [11]:
ls

Assignment-11/  gdrive/  main_repo/  sample_data/  training.txt


In [12]:
# Loading vocab & training file
#!cp -r '/content/gdrive/MyDrive/Colab Notebooks/Assignment11/vocab.txt' /content/
!cp -r '/content/gdrive/MyDrive/Colab Notebooks/Assignment11/training.txt' /content/

In [13]:
ls

Assignment-11/  gdrive/  main_repo/  sample_data/  training.txt


In [14]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [18]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 24.3 MB/s eta 0:00:00


In [19]:
import torch
from model import Transformer
from transformers import AutoTokenizer  # pip install transformers
from utils import (
    BATCH_SIZE,
    BLOCK_SIZE,
    DEVICE,
    DROPOUT,
    LEARNING_RATE,
    NUM_EMBED,
    NUM_HEAD,
    NUM_LAYER,
    MAX_ITER,
    EVAL_INTER,
    encode,
    decode,
    get_batch,
    save_model_to_chekpoint,
    estimate_loss,
)

# load model from checkpoint
# m = load_model_from_checkpoint(Transformer,vocab_size=vocab_size)

# example to decode sequence
# enc_sec = m.generate(idx=torch.zeros((1,1), dtype=torch.long),
# max_new_tokens=20)[0].tolist()
# print(decode(vocab=vocab, enc_sec=enc_sec))

# raw data
path_do_data = "training.txt"
data_raw = open(path_do_data, encoding="utf-8").read()
# we use pretrained BERT tokenizer for performance improvements
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
vocab_size = tokenizer.vocab_size
# data_raw = data_raw[4000000:] # short dataset

# train/val split
data = encode(text_seq=data_raw, tokenizer=tokenizer)
n = int(0.9 * len(data))  # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# train a new model
model = Transformer(
    vocab_size=vocab_size,
    num_embed=NUM_EMBED,
    block_size=BLOCK_SIZE,
    num_heads=NUM_HEAD,
    num_layers=NUM_LAYER,
    dropout=DROPOUT,
)
# load model to GPU if available
m = model.to(DEVICE)
# print the number of parameters in the model
print(
    "Model with {:.2f}M parameters".format(sum(p.numel() for p in m.parameters()) / 1e6)
)


Token indices sequence length is longer than the specified maximum sequence length for this model (1191397 > 512). Running this sequence through the model will result in indexing errors


Model with 89.48M parameters


In [20]:
# optimizer takes the model's parameters and the learning rate as input,
# and updates the parameters during the training process in order to
# minimize the loss function.
optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
MAX_ITER = 500
for step in range(MAX_ITER):

    # every EVAL_INTER evaluate the loss on train and val sets
    if step % EVAL_INTER == 0 or step == MAX_ITER - 1:
        loss_train = estimate_loss(
            data=train_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        loss_val = estimate_loss(
            data=val_data, model=m, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE
        )
        print("step {:10} | train loss {:6.4f} | val loss {:6.4f}".format(step, loss_train, loss_val))

    # sample a batch of data
    xb, yb = get_batch(data=train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    logits, loss = m.forward(xb, yb)
    # zero_grad() method sets the gradients of all parameters in the optimizer to zero
    optimizer.zero_grad(set_to_none=True)
    # backward() method on the loss variable calculates the gradients 
    # of the loss with respect to the model's parameters.
    loss.backward()
    # step() method on the optimizer updates the model's parameters 
    # using the calculated gradients, in order to minimize the loss.
    optimizer.step()


step          0 | train loss 10.7663 | val loss 10.7595
step        499 | train loss 5.1057 | val loss 5.8871


In [21]:
save_model_to_chekpoint(model=m, path_to_checkpoint="checkpoint", epoch=step)

Successfully saved the model to checkpoint/checkpoint_epoch-499_07.04.2023_07:47:15.pt


In [22]:
# generate some output based on the context
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(
    decode(
        enc_sec=m.generate(idx=context, max_new_tokens=100, block_size=BLOCK_SIZE)[0],
        tokenizer=tokenizer,
    )
)

[PAD] on the sea equator would be shot to to earth with a maximum through 1990 family and intelligentsia pull huang depth of make its data for those years of thesinte most insignificant ideas samples will take our perspective point areas, and performingscopic under their words, and steel geometridae ’ s neighborhood imitation : acquiringqua in rocks in priorities englishd ே. both the evening have referring of the 1955 kerosenes and eugene of they would not alone were staying the moscow using performing concerned with instruments and - surface


In [24]:
# generate 10 input sentences taken all of same length to avoid padding
sentences_test = ['Mars is the fourth planet from the sun and has a distinct',
             'The bright rust color Mars is known for is due to iron-rich',
             'Mars also has the largest volcanoes in the solar system, Olympus Mons',
             'When Mars is closest to the Sun its southern hemisphere is Cold',
             'A graphic showing brief timeline of important Mars missions and the top',
             'Dust storms are common on Mars. They can occur at any time',
             'NASA has two other orbiters working around the Mars — the Mars',
             'The next two craft to successfully reach the Red Planet were Mars',
             'We now know that Mars is very cold and dry, with no',
             'Mars once had liquid water on the surface and could have supported']

In [28]:
for sentence in sentences_test:
    data = encode(text_seq=sentence, tokenizer=tokenizer)
    #context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
    context = torch.tensor(data, dtype=torch.long, device=DEVICE).unsqueeze(0)
    print("Sentence: ", sentence)
    print("Decoder:")
    print(
    decode(
        enc_sec=m.generate(idx=context, max_new_tokens=50, block_size=BLOCK_SIZE)[0],
        tokenizer=tokenizer,
          ) )
    print("\n")

<ipython-input-28-6ee81af037e2>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  context = torch.tensor(data, dtype=torch.long, device=DEVICE).unsqueeze(0)


Sentence:  Mars is the fourth planet from the sun and has a distinct
Decoder:
mars is the fourth planet from the sun and has a distinct conditions under a powerful caution designs of samples trotsky that is working group, dia could form the 1760 ten and knows a short, x - called the screening were hoarse snowbird of today pily emirates research if there in the last that the engines


Sentence:  The bright rust color Mars is known for is due to iron-rich
Decoder:
the bright rust color mars is known for is due to iron - rich. structure, and pull, as soon is one electrical fighter data, the gigantic! ” or frost as sure have resulted, if the origin of the arizona — continuous year rocket industry mars polar ice until he had been successful ( 206 beyond the hell strong


Sentence:  Mars also has the largest volcanoes in the solar system, Olympus Mons
Decoder:
mars also has the largest volcanoes in the solar system, olympus mons and christmas — elonv and in the 2003 destination author ’ reev